In [ ]:
import os
import pandas as pd
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AdamW,
    get_linear_schedule_with_warmup,
)
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(
    "meta-llama/Llama-2-7b-hf", num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

In [ ]:
# Check if the tokenizer has a padding token, if not, set one
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

# Load the dataset
data = pd.read_csv("../Dataset_with_Features/dataset_11430.csv")

# Separate features
text_data = data["url"]
numerical_data = data.drop(columns=["url", "label"])
scaler = StandardScaler()
X_num = scaler.fit_transform(numerical_data)

# Define a max_length for tokenization
max_length = 128  # Example value, you should choose based on your data

# Preprocess text data with a progress bar
print("Tokenizing text data...")
tokenized_texts = []
for text in tqdm(text_data.tolist()):
    tokenized_text = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
    )
    tokenized_texts.append(tokenized_text["input_ids"][0])

# Make sure to tokenize with padding
tokenized_data = tokenizer(
    text_data.tolist(),
    padding="max_length",
    truncation=True,
    max_length=128,
    return_tensors="pt",
)  # Adjust max_length as needed
X_text = tokenized_data["input_ids"]

# Labels
y = data["label"].apply(lambda x: 1 if x == "bad" else 0)

# Split dataset
X_train_text, X_test_text, X_train_num, X_test_num, y_train, y_test = train_test_split(
    X_text, X_num, y, test_size=0.2, random_state=42
)

# Create TensorDatasets
train_dataset = TensorDataset(
    X_train_text,
    torch.tensor(X_train_num, dtype=torch.float),
    torch.tensor(y_train.values, dtype=torch.long),
)
test_dataset = TensorDataset(
    X_test_text,
    torch.tensor(X_test_num, dtype=torch.float),
    torch.tensor(y_test.values, dtype=torch.long),
)

In [ ]:
# Define collate function
def collate_batch(batch):
    texts, nums, labels = zip(*batch)
    texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)
    nums = torch.stack(nums)
    labels = torch.tensor(labels)
    return texts, nums, labels

In [ ]:
# DataLoader and training setup
batch_size = 16
epochs = 6
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Training settings
device = torch.device("cpu")
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * epochs)

In [ ]:
# Paths for saving model, tokenizer, and checkpoint
model_save_path = "../Model/savedModel/savedModel_11430"
tokenizer_save_path = "../Model/savedTokenizer/savedModel_11430"
checkpoint_path = "../Model/checkpoint.pth"

# Function to save a checkpoint
def save_checkpoint(epoch, model, optimizer, scheduler):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()
    }
    torch.save(checkpoint, checkpoint_path)
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(tokenizer_save_path)

# Function to load a checkpoint
def load_checkpoint(model, optimizer, scheduler):
    if os.path.isfile(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        return checkpoint['epoch']
    else:
        return 0

# Load checkpoint if available
start_epoch = load_checkpoint(model, optimizer, scheduler)

# Training loop
for epoch in range(start_epoch, epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}/{epochs}")

    for step, (texts, nums, labels) in progress_bar:
        texts = texts.to(device)
        labels = labels.to(device)

        model.zero_grad()
        outputs = model(texts, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        progress_bar.set_postfix({"loss": total_loss / (step + 1)})

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} | Loss: {average_loss}")

    # Save checkpoint after each epoch
    save_checkpoint(epoch + 1, model, optimizer, scheduler)

In [ ]:
# Evaluation loop
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for texts, nums, labels in tqdm(test_loader, desc='Evaluating', leave=False):
        texts = texts.to(device)
        labels = labels.to(device)
        nums = nums.to(device)

        outputs = model(texts, labels=labels)
        logits = outputs.logits
        _, predicted_labels = torch.max(logits, dim=1)
        
        predictions.extend(predicted_labels.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')

metrics_df = pd.DataFrame([[accuracy, precision, recall, f1]], columns=["Accuracy", "Precision", "Recall", "F1 Score"])
print(metrics_df)
metrics_df.to_csv("Evaluation_dataset_20000.csv", index=False)